In [2]:
import nltk
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [3]:
from nltk.corpus import wordnet as wn
food = wn.synset('food.n.02')
food_list = list(set([w for s in food.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))

In [3]:
len(food_list)

1621

In [5]:
print(food_list)

['loaf_sugar', 'snap', 'gherkin', 'bird', 'Golden_Delicious', 'edda', 'baguette', 'head_cabbage', 'berry', 'linguica', 'surinam_cherry', "lamb's-quarter", 'Cortland', 'rib', 'genoise', 'green_soybean', 'blackberry', 'river_pear', 'hoecake', 'convenience_food', 'hot_cake', 'coffee_ring', 'chevre', 'mad_apple', 'grouse', 'hot_dog', 'goat_cheese', 'muscat', 'cut', 'frozen_yogurt', 'lichee', 'leaf_lettuce', 'Spanish_mackerel', 'feijoa', 'foreshank', 'matzoh', 'vegetable', 'coffee_cake', 'jak', 'largemouth_bass', 'freshwater_bass', 'porc', 'petit_four', 'cowpea', 'sour_orange', 'soft_roll', 'wienerwurst', 'pandowdy', 'wintergreen', 'spoon_bread', 'apple', 'crape', 'pickerel', 'baking_chocolate', 'Madeira_sponge', 'plumcot', 'Belgian_waffle', 'bran_muffin', 'mussel', 'black_currant', 'periwinkle', "bullock's_heart", 'cruciferous_vegetable', 'Yorkshire_pudding', 'muskellunge', 'dark_bread', 'fresh_bean', 'aitchbone', 'carp', 'corn_muffin', 'whitebait', 'pattypan_squash', 'mutton_chop', 'blint

In [5]:
# Remove pucntutation and lowercase
food_list = [food_item.lower().replace('_', ' ') for food_item in food_list]

In [6]:
food_list[:10]

['cut of veal',
 'cornmeal mush',
 'chips',
 'haricot vert',
 'sausage meat',
 'spanish onion',
 'lake herring',
 'ash cake',
 'gigot',
 'bitter orange']

In [6]:
from huggingface_hub import login
login()

In [7]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset('imagenet-1k', split='train', streaming=True, trust_remote_code=True)

In [9]:
imagenet_classes = dataset.features['label'].names
imagenet_classes

['tench, Tinca tinca',
 'goldfish, Carassius auratus',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'tiger shark, Galeocerdo cuvieri',
 'hammerhead, hammerhead shark',
 'electric ray, crampfish, numbfish, torpedo',
 'stingray',
 'cock',
 'hen',
 'ostrich, Struthio camelus',
 'brambling, Fringilla montifringilla',
 'goldfinch, Carduelis carduelis',
 'house finch, linnet, Carpodacus mexicanus',
 'junco, snowbird',
 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 'robin, American robin, Turdus migratorius',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'water ouzel, dipper',
 'kite',
 'bald eagle, American eagle, Haliaeetus leucocephalus',
 'vulture',
 'great grey owl, great gray owl, Strix nebulosa',
 'European fire salamander, Salamandra salamandra',
 'common newt, Triturus vulgaris',
 'eft',
 'spotted salamander, Ambystoma maculatum',
 'axolotl, mud puppy, Ambystoma mexicanum',
 'bullfrog, Rana catesbeiana',
 'tree frog, tree-f

In [10]:
# Filter only food-related classes
food_classes = {i: name for i, name in enumerate(imagenet_classes) if any(food in name.lower() for food in food_list)}
food_classes


{1: 'goldfish, Carassius auratus',
 2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 4: 'hammerhead, hammerhead shark',
 5: 'electric ray, crampfish, numbfish, torpedo',
 8: 'hen',
 12: 'house finch, linnet, Carpodacus mexicanus',
 13: 'junco, snowbird',
 14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 18: 'magpie',
 25: 'European fire salamander, Salamandra salamandra',
 32: 'tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui',
 40: 'American chameleon, anole, Anolis carolinensis',
 46: 'green lizard, Lacerta viridis',
 47: 'African chameleon, Chamaeleo chamaeleon',
 49: 'African crocodile, Nile crocodile, Crocodylus niloticus',
 50: 'American alligator, Alligator mississipiensis',
 52: 'thunder snake, worm snake, Carphophis amoenus',
 53: 'ringneck snake, ring-necked snake, ring snake',
 54: 'hognose snake, puff adder, sand viper',
 55: 'green snake, grass snake',
 64: 'green mamba',
 68: 'sidewinder, horned r

In [11]:
not_food_ids = [
    1, 2, 4, 5, 8, 12, 13, 14, 18, 25, 32, 40, 46, 47, 49, 50, 52, 53, 54, 55,
    64, 68, 70, 80, 82, 83, 84, 85, 86, 92, 94, 99, 102, 103, 107, 109, 111, 113,
    117, 118, 119, 120, 121, 122, 123, 124, 125, 136, 137, 141, 143, 147, 153,
    161, 169, 190, 200, 209, 224, 227, 229, 230, 235, 238, 239, 245, 248, 251,
    252, 254, 256, 275, 298, 300, 301, 302, 303, 304, 305, 306, 313, 322, 323,
    324, 325, 326, 327, 329, 330, 331, 332, 333, 334, 339, 346, 376, 386, 390,
    391, 393, 395, 396, 397, 428, 431, 432, 445, 448, 461, 467, 471, 476, 477,
    479, 480, 498, 499, 503, 513, 520, 534, 537, 542, 580, 586, 587, 588, 598,
    607, 613, 619, 629, 630, 632, 644, 678, 679, 684, 694, 713, 718, 723, 726,
    729, 731, 739, 750, 752, 758, 762, 773, 778, 786, 798, 811, 821, 822, 829,
    837, 854, 856, 859, 874, 886, 891, 897, 905, 921, 923, 973, 975, 991, 996, 998
]


In [12]:
to_remove = [key for key in not_food_ids if key in food_classes]

In [13]:
for key in to_remove:
    del food_classes[key]

In [14]:
food_classes

{930: 'French loaf',
 931: 'bagel, beigel',
 932: 'pretzel',
 933: 'cheeseburger',
 934: 'hotdog, hot dog, red hot',
 935: 'mashed potato',
 936: 'head cabbage',
 937: 'broccoli',
 938: 'cauliflower',
 939: 'zucchini, courgette',
 940: 'spaghetti squash',
 941: 'acorn squash',
 942: 'butternut squash',
 943: 'cucumber, cuke',
 944: 'artichoke, globe artichoke',
 945: 'bell pepper',
 946: 'cardoon',
 947: 'mushroom',
 948: 'Granny Smith',
 949: 'strawberry',
 950: 'orange',
 951: 'lemon',
 952: 'fig',
 953: 'pineapple, ananas',
 954: 'banana',
 955: 'jackfruit, jak, jack',
 956: 'custard apple',
 957: 'pomegranate',
 960: 'chocolate sauce, chocolate syrup',
 962: 'meat loaf, meatloaf',
 963: 'pizza, pizza pie',
 964: 'potpie'}

In [15]:
len(food_classes)

32

In [16]:
# Filter out food-related classes
non_food_classes = {i: name for i, name in enumerate(imagenet_classes) if not any(food in name.lower() for food in food_list)}
non_food_classes

{0: 'tench, Tinca tinca',
 3: 'tiger shark, Galeocerdo cuvieri',
 6: 'stingray',
 7: 'cock',
 9: 'ostrich, Struthio camelus',
 10: 'brambling, Fringilla montifringilla',
 11: 'goldfinch, Carduelis carduelis',
 15: 'robin, American robin, Turdus migratorius',
 16: 'bulbul',
 17: 'jay',
 19: 'chickadee',
 20: 'water ouzel, dipper',
 21: 'kite',
 22: 'bald eagle, American eagle, Haliaeetus leucocephalus',
 23: 'vulture',
 24: 'great grey owl, great gray owl, Strix nebulosa',
 26: 'common newt, Triturus vulgaris',
 27: 'eft',
 28: 'spotted salamander, Ambystoma maculatum',
 29: 'axolotl, mud puppy, Ambystoma mexicanum',
 30: 'bullfrog, Rana catesbeiana',
 31: 'tree frog, tree-frog',
 33: 'loggerhead, loggerhead turtle, Caretta caretta',
 34: 'leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea',
 35: 'mud turtle',
 36: 'terrapin',
 37: 'box turtle, box tortoise',
 38: 'banded gecko',
 39: 'common iguana, iguana, Iguana iguana',
 41: 'whiptail, whiptail lizard',
 42: 'aga

In [17]:
# manually removing values that are food form non_food_classes filtering by keys
keys_to_remove =  [928, 929, 961, 965, 966, 967, 969, 987]

In [18]:
for key in keys_to_remove:
    if key in non_food_classes:
        food_classes[key] = non_food_classes[key]

In [19]:
len(food_classes)

40

In [20]:
import json
filtered_food_json_path = 'food_classes.json'
with open(filtered_food_json_path, 'w') as f:
    json.dump(food_classes, f, indent=4)

In [21]:
for key in keys_to_remove:
    if key in non_food_classes:
        del non_food_classes[key]

In [22]:
filtered_json_path = 'non_food_classes.json'
with open(filtered_json_path, 'w') as f:
    json.dump(non_food_classes, f, indent=4)

In [23]:
len(non_food_classes)

786

In [24]:
with open('non_food_classes.json', 'r') as f:
    non_food_classes = json.load(f)
import random
# Convert keys to integers
non_food_classes = {int(k): v for k, v in non_food_classes.items()}

# Define the number of classes and images per class
num_classes = 200
images_per_class = 50

# Randomly select classes
non_food_selected_classes = random.sample(non_food_classes.keys(), num_classes)

In [25]:
print(non_food_selected_classes)

[894, 971, 77, 455, 171, 668, 681, 478, 234, 114, 285, 823, 526, 286, 295, 148, 789, 139, 424, 379, 841, 649, 815, 101, 382, 728, 581, 658, 903, 640, 93, 37, 405, 648, 904, 149, 268, 576, 839, 328, 853, 311, 589, 450, 249, 259, 657, 852, 291, 421, 231, 784, 347, 530, 848, 879, 388, 911, 357, 96, 861, 918, 475, 422, 675, 797, 609, 202, 602, 289, 791, 154, 582, 447, 452, 783, 735, 756, 495, 601, 917, 599, 290, 794, 730, 995, 880, 927, 676, 31, 436, 413, 378, 583, 69, 222, 745, 308, 686, 706, 805, 309, 71, 152, 652, 253, 536, 733, 912, 460, 219, 532, 51, 743, 507, 90, 419, 568, 98, 144, 651, 349, 551, 877, 287, 830, 721, 594, 980, 206, 361, 868, 65, 496, 504, 74, 337, 548, 250, 160, 615, 702, 699, 100, 570, 366, 878, 469, 156, 781, 703, 226, 240, 775, 696, 223, 604, 554, 855, 407, 592, 804, 207, 825, 720, 867, 394, 372, 201, 444, 765, 33, 440, 667, 368, 976, 353, 674, 545, 369, 466, 884, 518, 348, 585, 989, 834, 627, 555, 502, 970, 221, 29, 788, 635, 228, 23, 843, 10, 559]


In [26]:
import os
# Create a directory to save the images
output_non_food_dir = 'imagenet_not_food_images'
os.makedirs(output_non_food_dir, exist_ok=True)

In [35]:
import os
from PIL import Image

def preprocess_and_save_images(dataset, selected_classes, output_dir, images_per_class, classes_dict):
    class_image_counts = {class_id: 0 for class_id in selected_classes}
    
    def save_image(image, class_id):
        class_name = classes_dict[class_id]
        class_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        img_path = os.path.join(class_dir, f"{class_image_counts[class_id]}.jpg")
        
        # Convert image to RGB if it is grayscale
        if image.mode != 'RGB':  # Grayscale image or any other mode
            image = image.convert("RGB")
        
        image.save(img_path)
        class_image_counts[class_id] += 1
        print(f"Saved {img_path}")
    
    for example in dataset:
        label = example['label']
        if label in selected_classes and class_image_counts[label] < images_per_class:
            image = Image.fromarray(example['image'].numpy())
            save_image(image, label)
        
        if all(count >= images_per_class for count in class_image_counts.values()):
            break

In [28]:
preprocess_and_save_images(dataset, non_food_selected_classes, output_non_food_dir, images_per_class, non_food_classes)

Saved imagenet_not_food_images\comic book\0.jpg
Saved imagenet_not_food_images\parallel bars, bars\0.jpg
Saved imagenet_not_food_images\trilobite\0.jpg
Saved imagenet_not_food_images\cocker spaniel, English cocker spaniel, cocker\0.jpg
Saved imagenet_not_food_images\tape player\0.jpg
Saved imagenet_not_food_images\baboon\0.jpg
Saved imagenet_not_food_images\rocking chair, rocker\0.jpg
Saved imagenet_not_food_images\oil filter\0.jpg
Saved imagenet_not_food_images\Pekinese, Pekingese, Peke\0.jpg
Saved imagenet_not_food_images\loggerhead, loggerhead turtle, Caretta caretta\0.jpg
Saved imagenet_not_food_images\ruddy turnstone, Arenaria interpres\0.jpg
Saved imagenet_not_food_images\thatch, thatched roof\0.jpg
Saved imagenet_not_food_images\soccer ball\0.jpg
Saved imagenet_not_food_images\grasshopper, hopper\0.jpg
Saved imagenet_not_food_images\bubble\0.jpg
Saved imagenet_not_food_images\coffee mug\0.jpg
Saved imagenet_not_food_images\folding chair\0.jpg
Saved imagenet_not_food_images\screw

In [30]:
with open('food_classes.json', 'r') as f:
    food_classes = json.load(f)
import random
output_food_dir = 'imagenet_food_images'
os.makedirs(output_food_dir, exist_ok=True)
# Convert keys to integers
food_classes = {int(k): v for k, v in food_classes.items()}

# Define the number of classes and images per class
num_classes = 40
images_per_class = 100

# Randomly select classes
food_selected_classes = random.sample(food_classes.keys(), num_classes)

In [38]:
preprocess_and_save_images(dataset, food_selected_classes, output_food_dir, images_per_class, food_classes)

Saved imagenet_food_images\zucchini, courgette\0.jpg
Saved imagenet_food_images\spaghetti squash\0.jpg
Saved imagenet_food_images\acorn squash\0.jpg
Saved imagenet_food_images\orange\0.jpg
Saved imagenet_food_images\spaghetti squash\1.jpg
Saved imagenet_food_images\eggnog\0.jpg
Saved imagenet_food_images\butternut squash\0.jpg
Saved imagenet_food_images\jackfruit, jak, jack\0.jpg
Saved imagenet_food_images\chocolate sauce, chocolate syrup\0.jpg
Saved imagenet_food_images\ice lolly, lolly, lollipop, popsicle\0.jpg
Saved imagenet_food_images\eggnog\1.jpg
Saved imagenet_food_images\bell pepper\0.jpg
Saved imagenet_food_images\potpie\0.jpg
Saved imagenet_food_images\pomegranate\0.jpg
Saved imagenet_food_images\zucchini, courgette\1.jpg
Saved imagenet_food_images\artichoke, globe artichoke\0.jpg
Saved imagenet_food_images\burrito\0.jpg
Saved imagenet_food_images\burrito\1.jpg
Saved imagenet_food_images\mushroom\0.jpg
Saved imagenet_food_images\artichoke, globe artichoke\1.jpg
Saved imagenet

d:\food_not_food\env\lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Saved imagenet_food_images\cauliflower\90.jpg
Saved imagenet_food_images\spaghetti squash\88.jpg
Saved imagenet_food_images\corn\88.jpg
Saved imagenet_food_images\head cabbage\91.jpg
Saved imagenet_food_images\meat loaf, meatloaf\88.jpg
Saved imagenet_food_images\Granny Smith\85.jpg
Saved imagenet_food_images\cauliflower\91.jpg
Saved imagenet_food_images\head cabbage\92.jpg
Saved imagenet_food_images\head cabbage\93.jpg
Saved imagenet_food_images\head cabbage\94.jpg
Saved imagenet_food_images\pineapple, ananas\77.jpg
Saved imagenet_food_images\Granny Smith\86.jpg
Saved imagenet_food_images\espresso\95.jpg
Saved imagenet_food_images\acorn squash\94.jpg
Saved imagenet_food_images\orange\99.jpg
Saved imagenet_food_images\acorn squash\95.jpg
Saved imagenet_food_images\cheeseburger\96.jpg
Saved imagenet_food_images\banana\91.jpg
Saved imagenet_food_images\cardoon\81.jpg
Saved imagenet_food_images\chocolate sauce, chocolate syrup\95.jpg
Saved imagenet_food_images\mushroom\96.jpg
Saved imagen